In [ ]:
# MODEL_NAME = "hopephoto/Qwen3-4B-Instruct-2507_q8"
# MODEL_NAME = "gemma3n:e4b"
MODEL_NAME = "gpt-oss:20b"

NUM_CTX = 8192
MAX_TOKENS = 2048

INPUT_DIRECTORY = "/content/drive/MyDrive/trans_in/"
OUTPUT_DIRECTORY = "/content/drive/MyDrive/trans_out/"

SYSTEM_PROMPT = """指示：
- 英語が入力された場合、日本語で出力してください。
- 出力は自然な母国語の話者のように表現してください。
- 出力形式は入力形式を忠実に再現してください。
- 冗長な表現や不適切な言い回しは適切に修正してください。
- JTF日本語標準スタイルガイドに従い、文法的に正しい自然な文章にし、句読点を適切に追加してください。
- 不明な言葉があれば質問してください。
- (最重要要件)出力をそのままコピーペーストするので不要な情報は含めず、結果の出力のみ行ってください。
"""

In [ ]:
# %env OLLAMA_NUM_PARALLEL=16
%env OLLAMA_FLASH_ATTENTION=1
%env OLLAMA_KV_CACHE_TYPE="q4_k_m"
%env OLLAMA_NO_HISTORY=1

In [ ]:
# Google ColabでGoogle Driveを使用する場合

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

def split_by_token_size(text, max_tokens):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(text)

    chunks = []
    current = []
    cur_len = 0
    for sent in doc.sents:
        sent_len = len(sent)
        if cur_len + sent_len > max_tokens and current:
            chunks.append(" ".join([t.text_with_ws for t in current]).strip())
            current = []
            cur_len = 0
        current.extend(sent)
        cur_len += sent_len
    if current:
        chunks.append(" ".join([t.text_with_ws for t in current]).strip())

    return chunks

In [ ]:
# Ollamaのインストール（必要であれば）

import sys

match sys.platform:
    case 'linux':
        !sudo apt update
        !sudo apt install -y pciutils
        !curl -fsSL https://ollama.com/install.sh | sh
    case 'darwin':
        !brew install ollama
    case _:
        # Windowsの場合、GUIからインストールしてください
        pass

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
!pip install ollama

In [ ]:
import os
import glob
import ollama

ollama.pull(MODEL_NAME)

def generate_file(file):
    with open(file, "r") as f:
        text = f.read()

    for i, chunk in enumerate(split_by_token_size(text, max_tokens=MAX_TOKENS)):
        result = ollama.generate(model=MODEL_NAME, prompt=text, system=SYSTEM_PROMPT, options={"num_ctx": NUM_CTX})
        print(str(result['response']))

        with open(os.path.join(OUTPUT_DIRECTORY, f"{os.path.splitext(os.path.basename(file))[0]}_generated_{i:03d}.txt"), "w", encoding="utf-8") as f:
            f.write(str(result['response']))

In [ ]:
file_list = sorted(glob.glob(os.path.join(INPUT_DIRECTORY, "*.txt")))
for file in file_list:
    generate_file(file)